This is just a little demo of how the code works. First import the module Data

In [1]:
import random
from Data import *

We get access to the Data class type, which can be used to load the data and provide many support function. We now create an instance of Data and load from a sample csv file. Finally, we normalize the value.

In [2]:
D = Data()
D.loadfile("sample.csv")
D.normalize()

For raw access:

In [3]:
D.data[0][1]

[0.15263157894736842,
 0.12055335968379449,
 0.716577540106952,
 0.48453608247422686,
 0.2608695652173913,
 0.606896551724138,
 0.5443037974683544,
 0.3018867924528301,
 0.6561514195583596,
 0.11689419795221842,
 0.3902439024390244,
 0.7289377289377289,
 0.2867332382310984]

To get a copy of a point:

In [4]:
t = D.get(4)

To denote unknown values, denote it as None. For a point p with missing values, we can update the unknown feature f by calling D.update(p, f):

In [5]:
t[1][10] = None
t[1][1] = None
t[1][5] = None
print("before",t)
D.update(t,10)
print("after:",t)

before (4, [0.16315789473684233, None, 0.6737967914438503, 0.7938144329896908, 0.1956521739130435, None, 0.2679324894514768, 0.5094339622641509, 0.2933753943217666, 0.11262798634812286, None, 0.7106227106227107, 0.20256776034236804])
after: (4, [0.16315789473684233, None, 0.6737967914438503, 0.7938144329896908, 0.1956521739130435, None, 0.2679324894514768, 0.5094339622641509, 0.2933753943217666, 0.11262798634812286, 0.7154471544715448, 0.7106227106227107, 0.20256776034236804])


You can get the binary vector of which features are known by calling bool_feature(p):

In [6]:
f = bool_feature(t[1])
print("bool features: ", f)
print("project p with f: ", len(proj(t[1],f)))

bool features:  [1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]
project p with f:  11


To partition the dataset into training and test sets, simply call D.split(r), r = ratio, which returns the test set and keeps D to be the training set

In [7]:
D_test = D.split(0.85)

print("D length:", len(D.data))
print("test length:", len(D_test.data))

D length: 152
test length: 26


Let's use k-means as our clustering algorithm. Calling D.cluster_K_means(k) will set the Data's clustering algorithm as k-means.

In [8]:
D.cluster_K_means(10)
D.clustering.cluster_centers_

[0.06578947368421052, 0.09868421052631579, 0.08552631578947369, 0.21052631578947367, 0.06578947368421052, 0.1118421052631579, 0.08552631578947369, 0.13815789473684212, 0.08552631578947369, 0.05263157894736842]


array([[0.49052632, 0.41719368, 0.62513369, 0.60309278, 0.47826087,
        0.20448276, 0.18797468, 0.2       , 0.22776025, 0.46331058,
        0.16341463, 0.08315018, 0.19614836],
       [0.3822807 , 0.11660079, 0.29661319, 0.35395189, 0.19130435,
        0.39954023, 0.33656821, 0.3245283 , 0.3022082 , 0.16751991,
        0.52737127, 0.57387057, 0.17318117],
       [0.6111336 , 0.3861356 , 0.57754011, 0.5777161 , 0.32608696,
        0.28328912, 0.09347614, 0.68069666, 0.34069401, 0.72309004,
        0.12070044, 0.14652015, 0.29496324],
       [0.70123355, 0.20930089, 0.63987299, 0.38498711, 0.42730978,
        0.66023707, 0.56711498, 0.33726415, 0.49201498, 0.37987948,
        0.50940041, 0.62190934, 0.66418509],
       [0.25815789, 0.2013834 , 0.58983957, 0.60154639, 0.16956522,
        0.31551724, 0.30021097, 0.71509434, 0.3318612 , 0.12303754,
        0.49674797, 0.48058608, 0.17453638],
       [0.31486068, 0.17728435, 0.54105064, 0.54244997, 0.25959079,
        0.60851927, 0.49776

In [9]:
r = random.randint(0,130)
print("r:",r)
t = D.get(r)
p = t[1]
print("t:", t)

print(D.score(t))
for i in range(12):
    print("projecting out dim:", i)
    p[i] = None
    print("score:", D.score(t))
    D.update(t,i)

r: 92
t: (92, [0.46315789473684194, 0.3814229249011858, 0.5989304812834224, 0.5876288659793815, 0.45652173913043476, 0.1724137931034483, 0.2151898734177215, 0.2075471698113207, 0.2681388012618297, 0.8122866894197953, 0.0, 0.07326007326007325, 0.14407988587731813])
0.0
projecting out dim: 0
score: 0.1414213562373095
projecting out dim: 1
score: 0.2
projecting out dim: 2
score: 0.1414213562373095
projecting out dim: 3
score: 0.2
projecting out dim: 4
score: 0.282842712474619
projecting out dim: 5
score: 0.1414213562373095
projecting out dim: 6
score: 0.0
projecting out dim: 7
score: 0.31622776601683794
projecting out dim: 8
score: 0.0
projecting out dim: 9
score: 0.2449489742783178
projecting out dim: 10
score: 0.282842712474619
projecting out dim: 11
score: 0.3741657386773941


Now, we compute the reward of state/action (s1,a) -> s2 as score(s1) - score(s2). This reward is positive
if score(s2) < score(s1); this is obviously desirable since higher score means "worse" state. 

In [10]:
# set N random features None
N = 13
t = D.get(r)
for i in range(N):
    while True:
        r = random.randint(0,12)
        if t[1][r] != None:
            break
    t[1][r] = None
print("t:",t)
print("score:", D.score(t))

print("\nREWARDS FOR A SINGLE ACTION")
for i in range(len(t[1])):
    if t[1][i] == None:
        print("reward of updating feature", i)
        print("reward:", D.reward(t,i))
        print("")

t: (92, [None, None, None, None, None, None, None, None, None, None, None, None, None])
score: 1.2

REWARDS FOR A SINGLE ACTION
reward of updating feature 0
reward: 0.6481693154528939

reward of updating feature 1
reward: 0.6309506430300089

reward of updating feature 2
reward: 0.1375947510695077

reward of updating feature 3
reward: 0.4365957592160836

reward of updating feature 4
reward: 0.20836247121870155

reward of updating feature 5
reward: 1.0592953899263313

reward of updating feature 6
reward: 0.6312615171320209

reward of updating feature 7
reward: -0.06805612395254101

reward of updating feature 8
reward: 0.55378375008549

reward of updating feature 9
reward: 0.4052337860397278

reward of updating feature 10
reward: 0.3473403650795953

reward of updating feature 11
reward: 1.1015196927418982

reward of updating feature 12
reward: 0.24222051018559565



If we want a list of actions to take, we can call D.actions(t), while D.next_state(state, action) will return the next state
with the action applied.

In [11]:
print("available actions:", D.actions(t))
print("next state with first available action applied:", D.next_state(t,D.actions(t)[0]))

available actions: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
next state with first available action applied: (92, [0.46315789473684194, None, None, None, None, None, None, None, None, None, None, None, None])
